In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from tensorflow import keras
from tensorflow.python.keras.models import load_model
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from keras.src.legacy.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.layers import Embedding,LSTM,Dense,Dropout
from keras.models import Sequential
from keras.utils import to_categorical

2024-10-08 15:56:12.934657: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-08 15:56:13.670284: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model = load_model('model.h5')

2024-10-08 15:56:17.946439: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-08 15:56:17.971684: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-08 15:56:17.974535: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
ps = PorterStemmer()
dict_size = 5000

In [7]:
# test thử.np_utils
txt = ["Regular fast food eating linked to fertility issues in women."]

# tiền xử lí dữ liệu
cleaned = []
for i in range(0,len(txt)):
    msg = re.sub('[^a-zA-Z]',' ',txt[i])
    msg = msg.lower()
    msg = msg.split()
    msg = [ps.stem(words) for words in msg if not words in set(stopwords.words('english'))]
    msg = ' '.join(msg)
    cleaned.append(msg)

one_hot_mat = [one_hot(words,dict_size) for words in cleaned]
embedded_layer = pad_sequences(one_hot_mat,padding = 'pre',maxlen = 201)
embedded_layer

# predict
pred = model.predict(embedded_layer)
cat = ['Business','Science','Entertainment','Health']
print(pred, cat[np.argmax(pred)])

[[9.2130971e-01 7.4869275e-02 9.0735004e-04 2.9136417e-03]] Business


In [11]:
import requests
import json

# Base URL of the API
BASE_URL = 'http://localhost:2005'

# Function to test /list_label endpoint (GET request)
def test_list_labels():
    url = f"{BASE_URL}/list_label"
    response = requests.get(url)
    
    if response.status_code == 200:
        labels = response.json().get('labels', [])
        print(f"Labels: {labels}")
    else:
        print(f"Failed to list labels. Status code: {response.status_code}")

# Function to test /classify endpoint (POST request)
def test_classify_text(input_text):
    url = f"{BASE_URL}/classify"
    headers = {'Content-Type': 'application/json'}
    data = {"text": input_text}
    
    response = requests.post(url, headers=headers, data=json.dumps(data))
    
    if response.status_code == 200:
        result = response.json()
        print(f"Input Text: {result['text']}")
        print(f"Predicted Label: {result['predicted_label']}")
        print(f"Probability: {result['prob']}")
    else:
        print(f"Failed to classify text. Status code: {response.status_code}")

# Run the test functions
if __name__ == '__main__':
    print("Testing /list_label endpoint:")
    test_list_labels()
    
    print("\nTesting /classify endpoint with text 'I love this product!':")
    test_classify_text("I love this product!")
    
    print("\nTesting /classify endpoint with text 'This is terrible.':")
    test_classify_text("This is terrible.")


Testing /list_label endpoint:
Labels: ['Business', 'Science', 'Entertainment', 'Health']

Testing /classify endpoint with text 'I love this product!':
Input Text: I love this product!
Predicted Label: Science
Probability: 0.7794768810272217

Testing /classify endpoint with text 'This is terrible.':
Input Text: This is terrible.
Predicted Label: Science
Probability: 0.37550053000450134


# Tools and Resources
OS: Ubuntu 22.04

Framework:
- Tensorflow 2.13.1
- NLTK
- Flask

Dataset: [News Aggregator Data Set](https://archive.ics.uci.edu/ml/datasets/News+Aggregator)

# Contribution
## Training model
Check out `training_notebook.ipynb` to see the preprocessing method and the architecture of model

## Run Flask server
Run app at `http://localhost:2005`:
```bash
python3 app.py
```

Test application:
```bash
python3 test_app.py
```

Results:
```bash
Testing /list_label endpoint:
Labels: ['Business', 'Science', 'Entertainment', 'Health']

Testing /classify endpoint with text 'I love this product!':
Input Text: An apple a day keeps doctor away.
Predicted Label: Health
Probability: 0.40282243490219116
```